In [1]:
# Install the packages
! pip3 install --upgrade google-cloud-aiplatform \
                         google-cloud-storage

# Install the packages
! pip install google-cloud-vision



INFO: pip is looking at multiple versions of google-api-core[grpc] to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 22.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.5/126.5 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 11.1 MB/s eta 0:00:00
  Attempting uninstall: google-api-core
    Found existing installation: google-api-core 1.34.1
    Uninstalling google-api-core-1.34.1:
      Successfully uninstalled google-api-core-1.34.1
  Attempting uninstall: google-cloud-storage
    Found existing installation: google-cloud-storage 2.14.0
    Uninstalling google-cloud-storage-2.14.0:
      Successfully uninstalled google-cloud-storage-2.14.0
  Attempting uninstall: google-cloud-aiplatform
    Found existing installation: google-cloud-aiplatform 1.58.0
    Uninstalling google-cloud-aiplatform-1.58.0:
      Successfully uninstalled 

In [3]:
!pip install --upgrade google-cloud-videointelligence

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.0/245.0 kB 1.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 5.8 MB/s eta 0:00:00ta 0:00:01
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.19.6
    Uninstalling protobuf-3.19.6:
      Successfully uninstalled protobuf-3.19.6
  Attempting uninstall: google-cloud-videointelligence
    Found existing installation: google-cloud-videointelligence 1.16.3
    Uninstalling google-cloud-videointelligence-1.16.3:
      Successfully uninstalled google-cloud-videointelligence-1.16.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
apache-beam 2.46.0 requires grpcio!=1.48.0,<2,>=1.33.1, but you have grpcio 1.48.0 which is incompatible.
apache-beam 2.46.0 requires protobuf<4,>3.12.2, but you have protobuf 4.25.3 which is incompatible.
google-api-python-client 1

In [8]:
# Automatically restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

In [22]:
#set project info
PROJECT = !gcloud config get-value project
PROJECT_ID = PROJECT[0]
REGION = "us-central1"


In [23]:
import json
import os
video_directory = "SampleVideo"

video_names=[]
for file_name in os.listdir(video_directory):
    if  not file_name.startswith('.'):
        video_names.append(file_name)

video_paths = [os.path.join(video_directory, video_name) for video_name in video_names]


### Define function to detect explicit images

enable clound vision api before running this bit

In [11]:
# from google.cloud.videointelligence.v1beta import SafeSearchAnnotation


In [3]:
#this has not been testes
from google.cloud import videointelligence

def analyze_video_safe_search(gcs_uri):
    client = videointelligence.VideoIntelligenceServiceClient()
    
    # Configure the request
    config = videointelligence.SafeSearchDetectionConfig()
    context = videointelligence.VideoContext(
        safe_search_detection_config=config
    )
    operation = client.annotate_video(
        request={"input_uri": gcs_uri, "features": [videointelligence.Feature.SAFE_SEARCH_DETECTION], "video_context": context}
    )
    
    # Wait for the operation to complete
    result = operation.result(timeout=180)
    
    # Process the result
    annotation_results = result.annotation_results[0]
    for frame in annotation_results.safe_search_annotations:
        print(f"Time offset: {frame.time_offset.seconds}.{frame.time_offset.nanos // 1000000}s")
        print(f"Adult: {videointelligence.Likelihood(frame.adult)}")
        print(f"Violence: {videointelligence.Likelihood(frame.violence)}")
        print(f"Racy: {videointelligence.Likelihood(frame.racy)}")
        print(f"Medical: {videointelligence.Likelihood(frame.medical)}")
        print(f"")

In [24]:
#for now the api is not active just to save costs. I set everything to true
is_safe_values_cloud_vision=[True for i in range (len(video_paths))]
is_safe_values_cloud_vision

[True]

In [25]:
# Filter images by safety
video_names = [
    video_name
    for video_name, is_safe in zip(video_names, is_safe_values_cloud_vision)
    if is_safe
]
video_paths = [
    video_path
    for video_path, is_safe in zip(video_paths, is_safe_values_cloud_vision)
    if is_safe
]

In [ ]:
"contents": {
    "role": "USER",
    "parts": [
      {
        "fileData": {
          "fileUri": "FILE_URI1",
          "mimeType": "MIME_TYPE"
        }
      },
      {
        "text": "TEXT1"
      },
      {
        "fileData": {
          "fileUri": "FILE_URI2",
          "mimeType": "MIME_TYPE"
        }
      },
      {
        "text": "TEXT2"
      },
      {
        "fileData": {
          "fileUri": "FILE_URI3",
          "mimeType": "MIME_TYPE"
        }
      },
        
        "videoMetadata": {
            "startOffset": {
              "seconds": integer,
              "nanos": integer
            },
            "endOffset": {
              "seconds": integer,
              "nanos": integer
            }
        }
            
            
    ]
  }

In [14]:
items

{'text': 'You are an assistant tasked with summarizing videos for retrieval.         These summaries will be embedded and used to retrieve the raw video.         Give a concise summary of the video that is well optimized for retrieval.',
 'fileData': {'fileUri': 'gs://github-repo/embeddings/getting_started_embeddings/UCF-101-subset/BrushingTeeth/v_BrushingTeeth_g01_c02.mp4',
  'mimeType': 'video/MP4'},
 'videoMetadata': {'startOffset': {'seconds': 0, 'nanos': 0},
  'endOffset': {'seconds': 4, 'nanos': 0}}}

In [21]:
 safety_settings = {
        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_ONLY_HIGH,
        HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
        HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
        HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
    }

In [22]:
safety_settings

{<HarmCategory.HARM_CATEGORY_HATE_SPEECH: 1>: <HarmBlockThreshold.BLOCK_ONLY_HIGH: 3>,
 <HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: 2>: <HarmBlockThreshold.BLOCK_ONLY_HIGH: 3>,
 <HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: 4>: <HarmBlockThreshold.BLOCK_ONLY_HIGH: 3>,
 <HarmCategory.HARM_CATEGORY_HARASSMENT: 3>: <HarmBlockThreshold.BLOCK_ONLY_HIGH: 3>}

In [15]:
import requests

text= """You are an assistant tasked with summarizing videos for retrieval. \
        These summaries will be embedded and used to retrieve the raw video. \
        Give a concise summary of the video for the given videometadata that is well optimized for retrieval."""
video_file="gs://cloud-samples-data/vertex-ai-vision/highway_vehicles.mp4"
if 1==1:
 
        # Load video file
        video_bytes = None
 

        instance ={}
        items={}
       
        items["role"]= "USER"
        if text:
            items["text"]= text

        if video_file:             
            items["fileData"]= {
                          "fileUri": video_file,
                          "mimeType": "video/MP4"
                        }
            items["videoMetadata"]= {

                                "startOffset": {
                                  "seconds": 0,
                                  "nanos": 0
                                },
                                "endOffset": {
                                  "seconds": 4,
                                  "nanos": 0
                                }
            }
            #as the files are incomplete, for now set it fix
            video_duration=120
            instance={}
            instance["contents"]={"role":"USER",

                                  "Parts":[items],


              "generationConfig": {
                "temperature": 1,
                "topP": 0.95, 
                "maxOutputTokens": 8192 
              }
            }

          

        endpoint = (
           f"projects/{PROJECT_ID}/locations/{REGION}"
           "/publishers/google/models/gemini-1.5-flash@001"
        )

        #response =self.client.predict(endpoint=endpoint, instances=instances)
#client.predict(endpoint=endpoint, instances=instances)
       

In [16]:
endpoint

'projects/nine-quality-test/locations/us-central1/publishers/google/models/gemini-1.5-flash@001'

In [26]:
client = aiplatform.gapic.PredictionServiceClient  (
            client_options=client_options
        )
    

In [29]:
endpoint = (
           f"projects/{PROJECT_ID}/locations/us-central1/publishers/google/models/gemini-1.5-flash-001"
        )
    
api_regional_endpoint= "us-central1-aiplatform.googleapis.com" 
client_options = {"api_endpoint": api_regional_endpoint}
        
client = aiplatform.gapic.PredictionServiceClient  (
            client_options=client_options
        )
    

response =client.stream_generate_content (endpoint=endpoint, parts=[instance])

TypeError: PredictionServiceClient.stream_generate_content() got an unexpected keyword argument 'endpoint'

In [9]:
json.loads(schema)

{'description': 'A list of comic book recommendations',
 'startOffset': 'decimal',
 'endOffset': 'decimal',
 'chapterSummary': 'string'}

In [116]:
import json
schema = '''
{
  "description": "A list of chapters",
  "type": "array",
  "items": {
	"type":"object",
	"properties": {
		"startOffset": {
			"type":"integer"
		},
		"endOffset": {
			"type":"integer"
		},
		"chapterSummary": {
			"type":"string"
		}
	},
	"required": ["startOffset","endOffset","chapterSummary" ]
  }
}
'''
 

g=GenerationConfig(temperature=1, top_k=40,top_p=0.95,max_output_tokens=8192, response_mime_type='application/json',
	response_schema=json.loads(schema)) 

In [87]:
segments_to_process=120
intervals=16
video_duration=10*60
val=0

for val in range (segments_to_process,video_duration+segments_to_process,segments_to_process):
                offset={'start':prev, 'end':val}
                prev=val    
                print(offset)

{'start': 120, 'end': 120}
{'start': 120, 'end': 240}
{'start': 240, 'end': 360}
{'start': 360, 'end': 480}
{'start': 480, 'end': 600}


In [191]:
video_chapters=[]
segments_to_process=120
intervals=16
video_duration=10*60
prev=0
import time

        
        
video_file = "gs://github-repo/embeddings/getting_started_embeddings/UCF-101-subset/BrushingTeeth/v_BrushingTeeth_g01_c02.mp4"
video_file="gs://raw_nine_files/60MI23_33_A_HBB.mp4"
video_file="gs://raw_nine_files/60MI24_1_A_HBB.mp4"

#video_file="gs://github-repo/embeddings/getting_started_embeddings/UCF-101-subset/BrushingTeeth/v_BrushingTeeth_g01_c02.mp4"
startOffset=0
endOffset=120
intervals=16


for val in range (segments_to_process,video_duration+segments_to_process,segments_to_process):
                offset={'start':prev, 'end':val}
                prev=val    
                print(offset)
                 
                print('getting summaries..')
                
                chapters=get_video_summarycontent(video_file=video_file,startOffset=offset['start'],endOffset=offset['end'],intervals=intervals)
               
                for chapter in chapters:
                    video_chapters.append(
                        VideoChapter(
                            embedding=[1,2,3],
                            #self.get_summarycontent_embedding_from_text_embedding_model(text=chapter["chapterSummary"]).text_embedding,                           
                            start_offset_sec=chapter["startOffset"],
                            end_offset_sec=chapter["endOffset"],
                            summary=chapter["chapterSummary"]
                        )
                    )
                print("hereh are you chapters:")
                print()
                print(chapters)
                time.sleep(10) 

{'start': 0, 'end': 120}
getting summaries..
candidates {
  content {
    role: "model"
    parts {
      text: "```json\n[\n  {\n    \"startOffset\": 0,\n    \"endOffset\": 16,\n    \"chapterSummary\": \"A woman walks down a hallway wearing a sequined dress.\"\n  },\n  {\n    \"startOffset\": 16,\n    \"endOffset\": 32,\n    \"chapterSummary\": \"A woman in a green top speaks about how her childhood was difficult and that she had an opportunity to stand up for herself later in life.\"\n  },\n  {\n    \"startOffset\": 32,\n    \"endOffset\": 48,\n    \"chapterSummary\": \"The woman speaks about a man that she would babysit for, who was a serial pedophile. She describes how the police investigated her case.\"\n  },\n  {\n    \"startOffset\": 48,\n    \"endOffset\": 64,\n    \"chapterSummary\": \"A man named Scott Tudnem was a detective who worked on the woman\'s case and investigated other sex offenders. He felt she was deserving of his attention and help to put the man away.\"\n  },\n 

In [190]:
import yaml


class VideoChapter:
    """Chapters generated from video with offset times."""

    start_offset_sec: int
    end_offset_sec: int
    embedding: List[float]
    summary: str

    def __init__(
        self, start_offset_sec: int, end_offset_sec: int, embedding: List[float], summary: str
    ):
        """Creates a `VideoEmbedding` object.

        Args:
            start_offset_sec: Start time offset (in seconds) of generated embeddings.
            end_offset_sec: End time offset (in seconds) of generated embeddings.
            embedding: Generated embedding for interval.
        """
        self.start_offset_sec = start_offset_sec
        self.end_offset_sec = end_offset_sec
        self.embedding = embedding
        self.summary=summary
        

def get_video_summarycontent( text: str = None, video_file: str = None,startOffset: int=0, endOffset: int=120, intervals: int=16):
        
         '''
         Gimini Features for Video as per:https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/video-understanding
         gemini-1.5-flash, Gemini 1.5 Pro: 
         With audio: ~50 minutes
         Without audio: 60 minutes
         Maximum videos per prompt: 10
         
         Gemini 1.0 Pro Vision:
         Maximum video length: 2 minutes
         The maximum videos per prompt: 1
         Audio in the video is ignored.
         
         Gimini feature for Part as per:https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/inference#sample-requests
        '''
        

 
         generative_multimodal_model= GenerativeModel("gemini-1.5-flash-001")
         # Please only capture key events and highlights.
        
         video_description_prompt=f"""You are an assistant tasked with summarizing videos for retrieval.\
         These summaries will be embedded and used to retrieve the raw video.\
        Chapterize the video content by grouping the video content into chapters \
        with intervals of {intervals} seconds and providing a concise detail for each chapter that is well optimized for retrieval.\
        If there is a famous person like politician, celebrity or athlete, indicate their name and describe what they are famous for.\
        Describe important scenes in the video concisely.\
        If you are not sure about any info, please do not make it up. \
        Only consider video from {startOffset} seconds to {endOffset} seconds. Ignore analyzing the rest of video.\
        If it is the last chapter, set the endOffset to {endOffset} instead.\ 
        If a chapter includes prohibited content, set chapterSummary to "".\
        For result, follow JSON schema.<JSONSchema>{json.dumps(schema)}</JSONSchema>"
        """
                   
         #generation_config= GenerationConfig(temperature=1, top_k=40,top_p=0.95,max_output_tokens=8192) 
         generation_config=GenerationConfig(temperature=1, top_k=40,top_p=0.95,max_output_tokens=8192)#, response_mime_type='application/json',
	         # response_schema=json.loads(schema))  
         #for video, BLOCK_NONE gives error. So, have to set it to BLOCK_ONLY_HIGH
         safety_settings=  {
                    HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
                    HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_ONLY_HIGH,
                    HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
                    HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
                    }
         stream=False
        
        
         # Load the saved video as a Gemini Part Object
          
         contents=[GenerativeModelPart.from_uri(video_file,mime_type="video/mp4"),
                   video_description_prompt,]
        
        
         model_response = generative_multimodal_model.generate_content(
            contents,
            generation_config=generation_config,
            stream=stream,
            safety_settings=safety_settings, )
        
        
         response_list = []
         print(model_response)
        
         if str(model_response.prompt_feedback).strip()!='block_reason: PROHIBITED_CONTENT':
             response=model_response.text
   
             chapters=[]
             chapters_text=re.findall(r'\{[^{}]*\}',response )
             print(chapters_text)
             print('\n')
             for chapter in chapters_text:
                         if 'startOffset' in chapter:
                             chapter=chapter.replace('{','').replace('}','').strip()
                             chapters.append(
                                 {
                                    "startOffset":chapter.split(',')[0].replace('"startOffset":','').replace("{",'').strip(),
                                    "endOffset":chapter.split(',')[1].replace('"endOffset":','').strip(),
                                     "chapterSummary": ('[ST]'+chapter[chapter.index('"chapterSummary":'): len(chapter)-1].replace('"chapterSummary":','').replace("}",'').strip()).replace('[ST]"','')

                                 }
                             )
         else:
               chapters.append(
                                 {
                                    "startOffset":str(startOffset),
                                    "endOffset":str(endOffset),
                                     "chapterSummary": '***PROHIBITED_CONTENT***'

                                 }
               )
         return chapters

#def get_video_summarycontent( text: str = None, video_file: str = None):
# def get_video_summarycontent1( text: str = None, video_file: str = None,startOffset: int=0, endOffset: int=120, intervals: int=16):
#          '''
#          Gimini Features for Video as per:https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/video-understanding
#          gemini-1.5-flash, Gemini 1.5 Pro: 
#          With audio: ~50 minutes
#          Without audio: 60 minutes
#          Maximum videos per prompt: 10
         
#          Gemini 1.0 Pro Vision:
#          Maximum video length: 2 minutes
#          The maximum videos per prompt: 1
#          Audio in the video is ignored.
         
#          Gimini feature for Part as per:https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/inference#sample-requests
#         '''
        

 
#          generative_multimodal_model= GenerativeModel("gemini-1.5-flash-001")
#          # Please only capture key events and highlights.
        
#          video_description_prompt=f"""You are an assistant tasked with summarizing videos for retrieval.\
#          These summaries will be embedded and used to retrieve the raw video.\
#         Chapterize the video content by grouping the video content into chapters \
#         with intervals of {intervals} seconds and providing a concise detail for each chapter that is well optimized for retrieval.\
#         If there is a famous person like politician, celebrity or athlete, indicate their name and describe what they are famous for.\
#         Describe important scenes in the video concisely.\
#         If you are not sure about any info, please do not make it up. \
#         Only consider video from {startOffset} seconds to {endOffset} seconds. Ignore analyzing the rest of video.\
#         Return the result as per given schema:\
#         {schema}.\
#         If it is the last chapter, set the endOffset to {endOffset} instead.\  
#         """
                   
#          generation_config= GenerationConfig(temperature=1, top_k=40,top_p=0.95,max_output_tokens=8192) 
         
#         #for video, BLOCK_NONE gives error. So, have to set it to BLOCK_ONLY_HIGH
#          safety_settings=  {
#                     HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
#                     HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_ONLY_HIGH,
#                     HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
#                     HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
#                     }
#          stream=True
        
#          # Load the saved video as a Gemini Part Object
          
#          contents=[GenerativeModelPart.from_uri(video_file,mime_type="video/mp4"),
#                    video_description_prompt,]
        
        
#          res = generative_multimodal_model.generate_content(
#             contents,
#             generation_config=generation_config,
#             stream=stream,
#             safety_settings=safety_settings, )
        
        
#          response_list = []

#          for chunk in res:
#             try:
#                 response_list.append(chunk.text)
#             except Exception as e:
#                 print(
#                     "Exception occurred while calling gemini. Something is wrong. Lower the safety thresholds [safety_settings: BLOCK_NONE ] if not already done. -----",
#                     e,
#                 )
#                 response_list.append("****Exception occurred***:"+e)
#                 continue
                
#          response = ''.join(response_list)
#          response=response.replace('''{\n  "description": "A list of chapters",\n  "type": "array",\n  "items": {\n\t"type":"object",\n\t"properties": {\n\t\t"startOffset": {\n\t\t\t"type":"integer"\n\t\t},\n\t\t"endOffset": {\n\t\t\t"type":"integer"\n\t\t},\n\t\t"chapterSummary": {\n\t\t\t"type":"string"\n\t\t}\n\t},\n\t"required": ["startOffset","endOffset" ]\n  }\n},\n  ''','')
#          response=response[response.index('"items"'):len(response)-1]

#          chapters=[]
#          chapters_text=re.findall(r'\{[^{}]*\}',response )
#          print(chapters_text)
#          print('\n')
#          for chapter in chapters_text:
#                      chapter=chapter.replace('{','').replace('}','').strip()
#                      chapters.append(
#                          {
#                             "startOffset":chapter.split(',')[0].replace('"startOffset":','').replace("{",'').strip(),
#                             "endOffset":chapter.split(',')[1].replace('"endOffset":','').strip(),
#                              "chapterSummary": ('[ST]'+chapter[chapter.index('"chapterSummary":'): len(chapter)-1].replace('"chapterSummary":','').replace("}",'').strip()).replace('[ST]"','')
                        
#                          }
#                      )
#          return chapters
        

In [189]:
import re
video_file="gs://raw_nine_files/60MI23_33_A_HBB.mp4"
def get_video_summarycontent( video_file: str = None,startOffset: int=0, endOffset: int=120, intervals: int=16):
        
         '''
         Gimini Features for Video as per:https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/video-understanding
         gemini-1.5-flash, Gemini 1.5 Pro: 
         With audio: ~50 minutes
         Without audio: 60 minutes
         Maximum videos per prompt: 10
         
         Gemini 1.0 Pro Vision:
         Maximum video length: 2 minutes
         The maximum videos per prompt: 1
         Audio in the video is ignored.
         
         Gimini feature for Part as per:https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/inference#sample-requests
        '''
        

 
         generative_multimodal_model= GenerativeModel("gemini-1.5-flash-001")
         # Please only capture key events and highlights.
        
         video_description_prompt=f"""You are an assistant tasked with summarizing videos for retrieval.\
         These summaries will be embedded and used to retrieve the raw video.\
        Chapterize the video content by grouping the video content into chapters \
        with intervals of {intervals} seconds and providing a concise summary for each chapter that is well optimized for retrieval.\
        If there is a famous person like politician, celebrity or athlete, indicate their name and describe what they are famous for.\
        If you are not sure about any info, please do not make it up. \
        Only consider video from {startOffset} seconds to {endOffset} seconds. Ignore analyzing the rest of video.\
        Return the result in the JSON format with keys as follows : "startOffset","endOffset", "chapterSummary".\
        If it is the last chapter, set the endOffset to {endOffset} instead.\
       """

        
         generation_config= GenerationConfig(temperature=1, top_k=40,top_p=0.95,max_output_tokens=8192) 
         
        #for video, BLOCK_NONE gives error. So, have to set it to BLOCK_ONLY_HIGH
         safety_settings=  {
                    HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
                    HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_ONLY_HIGH,
                    HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
                    HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
                    }
         stream=True
        
         # Load the saved video as a Gemini Part Object
          
         contents=[GenerativeModelPart.from_uri(video_file,mime_type="video/mp4"),
                   video_description_prompt,]
        
        
         response = generative_multimodal_model.generate_content(
            contents,
            generation_config=generation_config,
            stream=stream,
            safety_settings=safety_settings, )
        
        
         response_list = []

         for chunk in response:
            try:
                response_list.append(chunk.text)
            except Exception as e:
                print(
                    "Exception occurred while calling gemini. Something is wrong. Lower the safety thresholds [safety_settings: BLOCK_NONE ] if not already done. -----",
                    e,
                )
                response_list.append("Exception occurred")
                continue
         response = "".join(response_list)
        
         chapters_text=re.findall(r"\{.*?\}", response.replace('\n',''))
         chapters=[]
         for chapter in chapters_text:
            chapters.append(yaml.safe_load(chapter))
 
         return response,chapters

### Defining encoding functions
Create an EmbeddingPredictionClient which encapsulates the logic to call the embedding API.

In [149]:
from vertexai.preview.generative_models import (
    Content,
    GenerationConfig,
    GenerationResponse,
    GenerativeModel,
    Image,
    Part ,
    HarmBlockThreshold,
    HarmCategory 
)

from google.cloud.aiplatform_v1beta1.types import (
    content as gapic_content_types,
)
gapic_content_types

<module 'google.cloud.aiplatform_v1beta1.types.content' from '/opt/conda/lib/python3.10/site-packages/google/cloud/aiplatform_v1beta1/types/content.py'>

In [151]:
dir(gapic_content_types.SafetySetting)

['HarmBlockMethod',
 'HarmBlockThreshold',
 '__abstractmethods__',
 '__annotations__',
 '__base__',
 '__bases__',
 '__basicsize__',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dictoffset__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__flags__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__instancecheck__',
 '__itemsize__',
 '__le__',
 '__lt__',
 '__module__',
 '__mro__',
 '__name__',
 '__ne__',
 '__new__',
 '__or__',
 '__prepare__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__ror__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasscheck__',
 '__subclasses__',
 '__subclasshook__',
 '__text_signature__',
 '__weakrefoffset__',
 'copy_from',
 'deserialize',
 'from_json',
 'meta',
 'mro',
 'pb',
 'serialize',
 'to_dict',
 'to_json',
 'wrap']

In [1]:
import base64
import time
import typing
import math

from google.cloud import aiplatform
from google.protobuf import struct_pb2

#libraries to generate image summaries
from vertexai.vision_models import Video
from vertexai.vision_models import VideoSegmentConfig
from vertexai.vision_models import MultiModalEmbeddingModel
from vertexai.language_models import TextEmbeddingModel
from vertexai.vision_models import Image as vision_model_Image
from vertexai.preview.generative_models import (
    Content,
    GenerationConfig,
    GenerationResponse,
    GenerativeModel,
    Image,
    Part as GenerativeModelPart,
    HarmBlockThreshold,
    HarmCategory,
    model_response.prompt_feedback.block_reason.value
)
from typing import Any, Dict, List, Literal, Optional, Union

from moviepy.editor import VideoFileClip

text_embedding_model = TextEmbeddingModel.from_pretrained("textembedding-gecko@latest")
#multimodal_embedding_model = MultiModalEmbeddingModel.from_pretrained(
 #   "multimodalembedding@001"
#)
intervals=120
max_duration=120#math.ceil(2719.04)#


class VideoEmbedding:
    """Embeddings generated from video with offset times."""

    __module__ = "vertexai.vision_models"

    start_offset_sec: int
    end_offset_sec: int
    embedding: List[float]

    def __init__(
        self, start_offset_sec: int, end_offset_sec: int, embedding: List[float]
    ):
        """Creates a `VideoEmbedding` object.

        Args:
            start_offset_sec: Start time offset (in seconds) of generated embeddings.
            end_offset_sec: End time offset (in seconds) of generated embeddings.
            embedding: Generated embedding for interval.
        """
        self.start_offset_sec = start_offset_sec
        self.end_offset_sec = end_offset_sec
        self.embedding = embedding
        
class EmbeddingResponse(typing.NamedTuple):
    text_embedding: typing.Sequence[float]
    video_embedding: typing.Sequence[VideoEmbedding]
       
        
def load_video_bytes(video_uri: str) -> bytes:
    """Load image bytes from a remote or local URI."""
    video_bytes = None
    if video_uri.startswith("http://") or video_uri.startswith("https://") or video_uri.startswith("gs://"):
        video_uri=video_uri.replace("gs://", "https://storage.googleapis.com/").replace(
        " ", "%20")
            
        response = requests.get(video_uri, stream=True)
        if response.status_code == 200:
            video_bytes = response.content
    else:
        video_bytes = open(video_uri, "rb").read()
        

    return video_bytes




def get_video_duration(video_uri):
  try:
    
    if video_uri.startswith("http://") or video_uri.startswith("https://") or video_uri.startswith("gs://"):
        video_uri=video_uri.replace("gs://", "https://storage.googleapis.com/").replace(
        " ", "%20")
        
    clip = VideoFileClip(video_uri)
    duration = clip.duration
    clip.close()  # Release resources
    return duration
  except OSError as e:
    if "moov atom not found" in str(e):
      print("Error: The video file seems to be corrupted or incomplete.")
      #To Do: fix this
      #fix the issue using 
      ##!MP4Box -inter 0  'drive/MyDrive/Colab Notebooks/60MI23_33_A_HBB.mp4'
      #for now
      return max_duration
    else:
      print(f"Error reading video file: {e}")
    return None

    
class EmbeddingPredictionClient:
    """Wrapper around Prediction Service Client."""

    def __init__(
        self,
        project: str,
        location: str = "us-central1",
        api_regional_endpoint: str = "us-central1-aiplatform.googleapis.com",
    ):
        client_options = {"api_endpoint": api_regional_endpoint}
        # Initialize client that will be used to create and send requests.
        # This client only needs to be created once, and can be reused for multiple requests.
        self.client = aiplatform.gapic.PredictionServiceClient(
            client_options=client_options
        )
        self.location = location
        self.project = project

    
    def get_embedding(self, text: str = None, video_file: str = None  ):
        if not text and not video_file:
            raise ValueError("At least one of text or video_file must be specified.")

 
        # Load video file
        video_bytes = None
        if video_file:           
            video_bytes = load_video_bytes(video_file)

        instance ={}
        if text:
            instance["text"] = text

        if video_bytes:             
                     
            encoded_content = base64.b64encode(video_bytes).decode("utf-8")
            instance["video"] = {
                    "bytesBase64Encoded": encoded_content # pylint: disable=protected-access
                }  # pylint: disable=protected-access
            #get video duration
            #as the files are incomplete, for now set it fix
            video_duration=math.ceil(get_video_duration(video_file))


        endpoint = (
           f"projects/{self.project}/locations/{self.location}"
           "/publishers/google/models/multimodalembedding@001"
        )

        #response =self.client.predict(endpoint=endpoint, instances=instances)

        text_embedding = None
        if text:
            instances = [instance]
            response =self.client.predict(endpoint=endpoint, instances=instances)
            text_emb_value = response.predictions[0]["textEmbedding"]
            text_embedding = [v for v in text_emb_value]

         
        video_embedding = None
        if video_bytes:
            video_embeddings = []  
            prev=0
            #iterate over the file and get embeddings of the whole file
            for val in range (intervals,video_duration+intervals,intervals):
                offset={'start':prev, 'end':val}
                prev=val    
                print(offset)
                video_segments=VideoSegmentConfig(start_offset_sec=offset['start'],end_offset_sec=offset['end'])

                if video_segments:
                    instance["video"]["videoSegmentConfig"] = {
                            "startOffsetSec": video_segments.start_offset_sec,
                            "endOffsetSec": video_segments.end_offset_sec,
                            "intervalSec": video_segments.interval_sec,
                        }
                
                instances = [instance]
                response =self.client.predict(endpoint=endpoint, instances=instances)
                for video_embedding in response.predictions[0].get("videoEmbeddings", []):
                    video_embeddings.append(
                        VideoEmbedding(
                            embedding=video_embedding["embedding"],
                            start_offset_sec=video_embedding["startOffsetSec"],
                            end_offset_sec=video_embedding["endOffsetSec"],
                        )
                    )
                # video_embeddings.append(
                #         VideoEmbedding(
                #             embedding=[1,2,3],
                #             start_offset_sec=video_segments.start_offset_sec,
                #             end_offset_sec=video_segments.end_offset_sec,
                #         )
                #     )

                
        return EmbeddingResponse (text_embedding=text_embedding, video_embedding=video_embeddings)
 
   
   
    def get_video_summarycontent(self, text: str = None, video_file: str = None):
        
         '''
         Gimini Features for Video as per:https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/video-understanding
         gemini-1.5-flash, Gemini 1.5 Pro: 
         With audio: ~50 minutes
         Without audio: 60 minutes
         Maximum videos per prompt: 10
         
         Gemini 1.0 Pro Vision:
         Maximum video length: 2 minutes
         The maximum videos per prompt: 1
         Audio in the video is ignored.
         
         Gimini feature for Part as per:https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/inference#sample-requests
        '''
        

 
         generative_multimodal_model= GenerativeModel("gemini-1.5-flash-001")
         # Please only capture key events and highlights.
         video_description_prompt="""You are an assistant tasked with summarizing videos for retrieval.\
         These summaries will be embedded and used to retrieve the raw video.\
        Chapterize the video content by grouping the video content into chapters \
        and providing a concise summary for each chapter that is well optimized for retrieval.\
        If there is a famous person like politician, celebrity or athlete, indicate their name and describe what they are famous for.\
        If you are not sure about any info, please do not make it up. \
        Return the result in the JSON format with keys as follows : "timecode", "chapterSummary"
        """

         generation_config= GenerationConfig(temperature=1, max_output_tokens=8192,TopK=40,TopP=0.95) 
        
         safety_settings=  {
                    HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
                    HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
                    HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
                    HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
                    }
         stream=True
        
         # Load the saved video as a Gemini Part Object
        
         video_for_gemini=GenerativeModelPart.from_uri(video_file,mimeType='video/mp4')
   
         model_input=[video_description_prompt, video_for_gemini]
        
       
        
         response = generative_multimodal_model.generate_content(
            model_input,
            generation_config=generation_config,
            stream=stream,
            safety_settings=safety_settings, )
        
        
         response_list = []

         for chunk in response:
            try:
                response_list.append(chunk.text)
            except Exception as e:
                print(
                    "Exception occurred while calling gemini. Something is wrong. Lower the safety thresholds [safety_settings: BLOCK_NONE ] if not already done. -----",
                    e,
                )
                response_list.append("Exception occurred")
                continue
         response = "".join(response_list)
 
         return response

    
    def get_summarycontent_embedding_from_text_embedding_model(self, text: str, return_array: Optional[bool] = False,) -> list:
        """
        Generates a numerical text embedding from a provided text input using a text embedding model.

        Args:
            text: The input text string to be embedded.
            return_array: If True, returns the embedding as a NumPy array.
                          If False, returns the embedding as a list. (Default: False)

        Returns:
            list or numpy.ndarray: A 768-dimensional vector representation of the input text.
                                   The format (list or NumPy array) depends on the
                                   value of the 'return_array' parameter.
        """

        #the given text is maximum 2048 token. If more, it has to be chunked.
        embeddings = text_embedding_model.get_embeddings([text])
        text_embedding = [embedding.values for embedding in embeddings][0]

        if return_array:
            text_embedding = np.fromiter(text_embedding, dtype=float)

        # returns 768 dimensional array
        return EmbeddingResponse(
            text_embedding=text_embedding, image_embedding=None
        )
    

### Create helper functions to process data in batches
Datasets can be large, so it's recommended to load a batch of data at a time into memory using a generator.

In [6]:
import time
from concurrent.futures import ThreadPoolExecutor
from typing import Callable, Generator, List,Tuple

from tqdm.auto import tqdm

#Number of API calls per second
API_IMAGES_PER_SECOND = 2

def generate_batches(
    inputs: List[str], batch_size: int
) -> Generator[List[str], None, None]:
    """
    Generator function that takes a list of strings and a batch size, and yields batches of the specified size.
    """
    for i in range(0, len(inputs), batch_size):
        yield inputs [i : i + batch_size] 



def encode_to_embeddings_chunked(
    process_function: Callable[[List[str]], List[Optional[List[float]]]],
    items: str,
    batch_size: int = 1,
) -> List[Optional[List[float]]]:
    """
    Function that encodes a list of strings into embeddings using a process function.
    It takes a list of strings and returns a list of optional lists of floats.
    The data is processed in chunks to prevent out-of-memory errors.
    """
    
    embeddings_list: List[Optional[List[float]]] = []

    # Prepare the batches using a generator
    batches = generate_batches(items, batch_size)
    
   
    seconds_per_job = batch_size / API_IMAGES_PER_SECOND

    with ThreadPoolExecutor() as executor:
        futures = []
        for batch in tqdm(batches, total=len(items) // batch_size, position=0):
            futures.append(executor.submit(process_function, batch))
            time.sleep(seconds_per_job)

        for future in futures:
            embeddings_list.extend(future.result())
    return embeddings_list

### Create functions that wrap embedding functions in try-except and retry logic
This particular embedding model can only process 1 image at a time, so inputs are validated to be equal to a length of 1.

In [7]:
import copy
from typing import List, Optional

import numpy as np
import requests
from tenacity import retry, stop_after_attempt

client = EmbeddingPredictionClient(project=PROJECT_ID)


# Use a retry handler in case of failure
@retry(reraise=True, stop=stop_after_attempt(3))
def encode_texts_to_embeddings_with_retry(text: List[str]) -> List[List[float]]:
    assert len(text) == 1

    try:
        return [client.get_embedding(text=text[0], image_file=None).text_embedding]
    except Exception:
        raise RuntimeError("Error getting embedding.")


def encode_texts_to_embeddings(text: List[str]) -> List[Optional[List[float]]]:
    try:
        return encode_texts_to_embeddings_with_retry(text=text)
    except Exception:
        return [None for _ in range(len(text))]


@retry(reraise=True, stop=stop_after_attempt(3))
def encode_videos_to_embeddings_with_retry(video_uris: List[str] ) -> List[List[float]]:
    assert len(video_uris) == 1

    try:
        return [
            client.get_embedding(text=None, video_file=video_uris[0] ).video_embedding
        ]
    except Exception as ex:
        print(ex)
        raise RuntimeError("Error getting embedding for video.")


def encode_videos_to_embeddings(video_uris: List[str] ) -> List[Optional[List[float]]]:
    try:
        
        return encode_videos_to_embeddings_with_retry(video_uris=video_uris )
    except Exception as ex:
        print(ex)
        return [None for _ in range(len(video_uris))]
    

@retry(reraise=True, stop=stop_after_attempt(3))
def encode_videos_to_summarycontent_with_retry(video_uris: List[str]) -> List[List[float]]:
    assert len(image_uris) == 1

    try:
        return [
            client.get_video_summarycontent(text=None, video_file=video_uris[0])
        ]
    except Exception as ex:
        print(ex)
        raise RuntimeError("Error getting summaries.")


def encode_videos_to_summarycontent(video_uris: List[str]) -> List[Optional[List[float]]]:
    try:
        return encode_videos_to_summarycontent_with_retry(video_uris=video_uris)
    except Exception as ex:
        print(ex)
        return [None for _ in range(len(video_uris))]
    
    
# Use a retry handler in case of failure
@retry(reraise=True, stop=stop_after_attempt(3))
def encode_summarycontent_to_embeddings_with_retry(text: List[str]) -> List[List[float]]:
    assert len(text) == 1

    try:
        return [client.get_summarycontent_embedding_from_text_embedding_model(text=text[0]).text_embedding]
    except Exception:
        raise RuntimeError("Error getting embedding for summary content.")


def encode_summarycontent_to_embeddings(text: List[str]) -> List[Optional[List[float]]]:
    try:
        return encode_summarycontent_to_embeddings_with_retry(text=text)
    except Exception:
        return [None for _ in range(len(text))]
    

### Create and save the embeddings in JSONL format
The data must be formatted in JSONL format, which means each embedding dictionary is written as an individual JSON object on its own line.

See more information in the docs at Input data format and structure.

Run the following code in the next available cells, to create a temporary file to store embeddings in JSON format.

In [9]:
import tempfile, shutil

# Create temporary file to write embeddings to
videoembeddings_file = tempfile.NamedTemporaryFile(suffix=".json", delete=False)

# Create temporary file to write summaries to
videosummaries_file = tempfile.NamedTemporaryFile(suffix=".json", delete=False)


### embedding file


### Video Embeddings in batch

In [85]:
video_paths_chunk

['SampleVideo/highway_vehicles.mp4']

In [10]:
import json

BATCH_SIZE = 1# this can be changed
 
           
with open(videoembeddings_file.name, "a") as ef:     
         for i in tqdm(range(0, len(video_paths), BATCH_SIZE)):#len(image_names)
            video_names_chunk = video_names[i : i + BATCH_SIZE]
            video_paths_chunk = video_paths[i : i + BATCH_SIZE]
            
            
            embeddings=[]
            video_summaries=[]
            #comment to prevent extra costs
            #********************************
            embeddings = encode_to_embeddings_chunked(
                    process_function=encode_videos_to_embeddings, items=video_paths_chunk)                 
            #********************************

            #comment to prevent extra costs
            #********************************
            #summaries = encode_to_embeddings_chunked(
                #process_function=encode_videos_to_summarycontent, items=video_paths_chunk
               #)

            #********************************
            #summaries=[' The image shows three people: Joe Biden, a young girl, and Hunter Biden. Joe Biden is smiling and wearing a dark suit. The young girl is smiling and wearing a white dress. Hunter Biden is smiling and wearing a dark suit. The background is a photo of the White House.','this is test']

            #comment to prevent extra costs
            #********************************
            #summaries_embeddings = encode_to_embeddings_chunked(
                 #process_function=encode_summarycontent_to_embeddings, items=summaries
            #)
            #summaries_embeddings=[[1,2],[1,2,3]]

            #********************************

            print(embeddings)

            # Append to file
            embeddings_formatted=[]
            for id,path,embedding in zip(video_names_chunk,video_paths_chunk,embeddings):
                for value in embedding:
                    if value.embedding is not None:
                        embeddings_formatted.append(  
                            json.dumps(
                                {
                                    "id": str(id), 
                                    "video path":str(path),
                                    "embedding": [str(value) for value in value.embedding],
                                    "start_offset_sec": value.start_offset_sec,
                                    "end_offset_sec": value.end_offset_sec 
                                }
                            )
                            + "\n"

                        )
            ef.writelines(embeddings_formatted)
        
 
        


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

{'start': 0, 'end': 120}
[[<vertexai.vision_models.VideoEmbedding object at 0x7ff06eff4df0>]]


### Create bucket and push embeddings into the bucket

In [ ]:

#set bucket info to create a bucket
BUCKET_URI = f"gs://artifacts-{PROJECT_ID}-unique"  # @param {type:"string"}
! gsutil mb -l {REGION} -p {PROJECT_ID} {BUCKET_URI}

In [ ]:
UNIQUE_FOLDER_NAME = "embeddings_results_NinePOC"


In [ ]:
#embeddings
EMBEDDINGS_INITIAL_URI = f"{BUCKET_URI}/{UNIQUE_FOLDER_NAME}/"
! gsutil cp {embeddings_file.name} {EMBEDDINGS_INITIAL_URI}

#summaries
EMBEDDINGS_INITIAL_URI = f"{BUCKET_URI}/{UNIQUE_FOLDER_NAME}/"
! gsutil cp {summaries_file.name} {EMBEDDINGS_INITIAL_URI}

In [89]:
#save the temp files in persistent disk
import tempfile, shutil
 
file_name = embeddings_file.name
embeddings_file.close()
shutil.copy(file_name, 'embeddings_file.json')

file_name = summaries_file.name
summaries_file.close()
shutil.copy(file_name, 'summaries_file.json')
 

'summaries_file.json'